In [1]:
# Generic OpenAI-compatible endpoint using ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
from langchain_ollama import OllamaEmbeddings 
from langchain_postgres.vectorstores import PGVector
  
import os
default_model_name = os.environ["OPENAI_MODEL"]
pg_connection = os.environ["PGVECTOR_CONNECTION_STRING"]
llm = ChatOpenAI(model=default_model_name,temperature=0)
embeddings_model = OllamaEmbeddings(model="nomic-embed-text")


In [2]:

from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda 

# Dua prompt
physics_template = """Anda adalah profesor fisika yang sangat pintar. Anda hebat dalam
    menjawab pertanyaan tentang fisika dengan cara ringkas dan mudah dipahami.
    Ketika Anda tidak tahu jawaban suatu pertanyaan, Anda mengakui bahwa Anda tidak tahu.

Ini sebuah pertanyaan:
{query}"""

math_template = """Anda adalah matematikawan yang sangat baik. Anda hebat dalam menjawab
    pertanyaan matematika. Anda sangat baik karena mampu memecah masalah
    sulit menjadi bagian-bagian komponennya, menjawab bagian-bagian komponen, lalu
    menggabungkannya untuk menjawab pertanyaan yang lebih luas.

Ini sebuah pertanyaan:
{query}"""

# Embed prompt
embeddings = embeddings_model
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Arahkan pertanyaan ke prompt

def prompt_router(query):
    # Embed pertanyaan
    query_embedding = embeddings.embed_query(query)
    # Hitung kesamaan
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    # Pilih prompt paling mirip dengan pertanyaan masukan
    most_similar = prompt_templates[similarity.argmax()]
    return PromptTemplate.from_template(most_similar)

semantic_router = (
    RunnableLambda(prompt_router)
    | llm
    | StrOutputParser()
)

print(semantic_router.invoke("Apa itu lubang hitam"))

Terima kasih atas pujian dan kepercayaannya! Namun, saya ingin sedikit meluruskan: meskipun saya bisa membantu menjawab berbagai jenis pertanyaan, termasuk matematika, **saya bukan seorang matematikawan**. Saya adalah asisten AI yang dirancang untuk memberikan informasi dan bantuan di berbagai bidang, termasuk sains, teknologi, dan matematika.

Sekarang, mari kita bahas pertanyaanmu:

---

### **Apa itu lubang hitam?**

Lubang hitam adalah **objek astronomi dengan gravitasi sangat kuat**, sehingga **tidak ada apa pun yang bisa lolos darinya**, bahkan **cahaya sekalipun**. Karena cahaya tidak bisa keluar, objek ini tampak "hitam" dan tidak dapat dilihat secara langsung — itulah sebabnya disebut *lubang hitam*.

Namun, lubang hitam dapat **dideteksi melalui pengaruh gravitasinya terhadap materi dan cahaya di sekitarnya**.

---

### **Bagian-bagian penting lubang hitam:**

1. **Horizon peristiwa (event horizon):**
   - Ini adalah **batas tak terlihat** di sekitar lubang hitam.
   - Setela

In [3]:
question = """Mengapa kode berikut tidak berfungsi:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

result.datasource

'python_docs'

In [4]:
from langchain_core.runnables import RunnableLambda
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logika di sini
        return "rantai untuk python_docs"
    else:
        ### Logika di sini
        return "rantai untuk js_docs"

full_chain = router | RunnableLambda(choose_route)
result = full_chain.invoke({"question": question})

result

'rantai untuk python_docs'